In [1]:
import torchvision.models as models
import torch as torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from utils.helper import CleanCuda, GetDevice

In [2]:
WEIGHTPATH = 'weights/resnet50.pth'

In [3]:
resnet50 = models.resnet50(pretrained=False)

/u/dw3zn/Repos/saliency_calibration/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/u/dw3zn/Repos/saliency_calibration/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [4]:
class CifarData():
    """
    """
    def __init__(self):
        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]
        self.train_transforms = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean, std),
            transforms.Resize([32, 32])
        ])
        self.val_transforms = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean, std),
            transforms.Resize([32, 32])
        ])
        
    def get_dataset(self):
        """
        Uses torchvision.datasets.ImageNet to load dataset.
        Downloads dataset if doesn't exist already.
        Returns:
             torch.utils.data.TensorDataset: trainset, valset
        """

        trainset = datasets.CIFAR100('datasets/CIFAR100/train/', train='True', transform=self.train_transforms,
                                     target_transform=None, download=True)
        valset = datasets.CIFAR100('datasets/CIFAR100/val/', train='False', transform=self.val_transforms,
                                   target_transform=None, download=True)

        return trainset, valset 
    
    
    def get_data_loader(self, batch_size=16):
        """
        Uses Class Object methods to generate
        torch dataloaders for train and val set
        
        param: batch_size: duh
        """
        trainset, valset = self.get_dataset()
        trainloader = DataLoader(trainset,
                            batch_size=batch_size,
                            shuffle=True,
                            num_workers=2,
                            )
        valloader = DataLoader(valset,
                               batch_size=1,
                               shuffle=True,)
        return trainloader, valloader

In [5]:
def train_model(model, optimizer, dataloader, num_epochs=1, 
         criterion=nn.CrossEntropyLoss(), 
         ):
    device = GetDevice()
    model.train()
    model = model.to(device)
    running_loss = 0
    for epoch in range(0, num_epochs):
        total_loss = 0
        for i, batch in enumerate(tqdm(dataloader)):
            imgs, labels = batch
            imgs = imgs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        if epoch == 0:
            running_loss = total_loss
        else:
            if(running_loss > total_loss):
                running_loss = total_loss
                torch.save(model.state_dict(), WEIGHTPATH)
        print(f"Training loss: {total_loss} in Epoch: {epoch+1}")
    return model

In [6]:
def evaluate(model, dataloader):
    device = GetDevice()
    num_correct = 0
    num_seen = 0
    model.load_state_dict(torch.load(WEIGHTPATH))
    model.eval()
    sm = nn.Softmax(dim = 1)
    for i, batch in enumerate(tqdm(dataloader)):
            imgs,  label = batch
            imgs = imgs.to(device)
            label = label.to(device)
            outputs = model(imgs)
            pred = sm(outputs)
            pred = torch.argmax(pred, dim=1).item()
            num_seen += len(label)
            if(pred == label.item()):
                num_correct += 1
            else:
                pass
    acc = (num_correct/num_seen) * 100
    print(f"Validation Accuracy: {acc:.2f} %")
    return acc

In [7]:
trainloader, valloader = CifarData().get_data_loader(2048)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting datasets/CIFAR100/train/cifar-100-python.tar.gz to datasets/CIFAR100/train/


  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting datasets/CIFAR100/val/cifar-100-python.tar.gz to datasets/CIFAR100/val/


In [8]:
optimizer = optim.Adam(resnet50.parameters(),
                      lr=0.0001, eps=1e-08,)
criterion = nn.CrossEntropyLoss()

In [9]:
_ = train_model(resnet50, optimizer, trainloader, 20, criterion)

There are 4 GPU(s) available.
Device name: NVIDIA GeForce RTX 2080 Ti


100%|███████████████████████████████████████████| 25/25 [00:10<00:00,  2.39it/s]


Training loss: 131.85928010940552 in Epoch: 1


100%|███████████████████████████████████████████| 25/25 [00:08<00:00,  2.79it/s]


Training loss: 113.65455293655396 in Epoch: 2


...

Training loss: 13.847102791070938 in Epoch: 18


100%|███████████████████████████████████████████| 25/25 [00:08<00:00,  2.78it/s]


Training loss: 10.194314062595367 in Epoch: 19


100%|███████████████████████████████████████████| 25/25 [00:09<00:00,  2.78it/s]


Training loss: 7.138386845588684 in Epoch: 20


In [10]:
evaluate(resnet50, valloader)

There are 4 GPU(s) available.
Device name: NVIDIA GeForce RTX 2080 Ti


100%|████████████████████████████████████| 50000/50000 [07:34<00:00, 110.06it/s]

Validation Accuracy: 99.17 %


99.166